In [1]:
# useing this variable for saving models and summaries
exp_name = "CNN1019"

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import pickle 

import sys
sys.path.append('../../../src')

In [3]:
import models.CNN_model as CNN_model
import training
import evaluation.nn_eval as nn_eval

## Loading synthetic data

In [4]:
#loading data
data_path= '/data2/data/zebrahim/synthetic_dataset/data_8192_1000_3_4_4_.03_.03_.2' + 'processed_data.p'

with open(data_path, 'r') as fin:
    data = pickle.load(fin)
    
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = np.expand_dims(data['train_gt'], axis=-1)
validation_gt = np.expand_dims(data['validation_gt'], axis=-1)
test_gt = np.expand_dims(data['test_gt'], axis=-1)

index_of_changes = data['index_of_changes']
 

In [5]:
#index of changes for test dataset
gt_test_idx_changes = index_of_changes[900:1000]

## Model

In [6]:
# Model parameters
n_variables = 12
learning_rate = 0.001
batch_size = 10

convolution_levels = 6    #number of convolution to have in the network
pooling_stride = [2, 2, 2, 2, 2, 1]
weights_shape = [[7, n_variables, 32],
                 [7, 32, 32],
                 [7, 32, 32],
                 [7, 32, 32],
                 [7, 32, 32],
                 [7, 32, 32]]
bias_shape = [32, 32, 32, 32, 32, 32]
activation = tf.nn.relu
n_classes = 1

##positive weight for weighted cross entropy
s = train_data.shape
resolution = np.prod(pooling_stride)
gt_effective = np.reshape(train_gt, (s[0]*(s[1]/resolution), resolution))
pos_weight = (1.0 - gt_effective.max(axis=1).mean()) / gt_effective.max(axis=1).mean()

In [7]:
print pos_weight

3.18601822124


In [8]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [9]:
summary_path = '../summary/' + exp_name 
checkpoint_path = '../model/' + exp_name + '/' + exp_name 

In [10]:
model = CNN_model.Model(n_variables = n_variables,
                        learning_rate = learning_rate,
                        n_classes = n_classes)

In [11]:
model.build_conv_net(pos_weight = pos_weight,
                     learning_rate = learning_rate,
                     convolution_levels = convolution_levels, 
                     pooling_stride = pooling_stride,
                     weights_shape = weights_shape,
                     bias_shape = bias_shape,
                     n_classes = n_classes,
                     activation = activation)

In [12]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = validation_data,
                    ground_truth_validation = validation_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0)

In [13]:
trainer.train(max_iter = 10000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Tarin Iter 0, auc= 0.219950, fmeasure_max= 0.398247, precision= 0.248632, recall= 1.000000
Train Random guess:  , auc= 0.245187fmeasure_max= 0.398122, precision= 0.248535, recall= 1.000000
Validation Iter 0, auc= 0.203155, fmeasure_max= 0.371219, precision= 0.227912, recall= 1.000000
Validation Random guess:  auc= 0.230797, fmeasure_max= 0.371392, precision= 0.228402, recall= 0.993139
Minibatch Loss= 1.081333
Tarin Iter 100, auc= 0.786594, fmeasure_max= 0.736645, precision= 0.770588, recall= 0.705566
Train Random guess:  , auc= 0.231087fmeasure_max= 0.357624, precision= 0.217748, recall= 1.000000
Validation Iter 100, auc= 0.774640, fmeasure_max= 0.763964, precision= 0.804554, recall= 0.727273
Validation Random guess:  auc= 0.213178, fmeasure_max= 0.371529, precision= 0.228235, recall= 0.998285
Minibatch Loss= 0.739191
Tarin Iter 200, auc= 0.767953, fmeasure_max= 0.729890, precision= 0.773788, recall= 0.690705
Train Random guess:  , auc= 0.233223fmeasure_max= 0.396716, precision= 0.2494

In [ ]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

In [ ]:
#load the model
saver = tf.train.Saver()
saver.restore(sess, "../models/model" + exp_name + "/model")

In [14]:
#This is the model evaluation on test dataset
probability_of_pos= nn_eval.evaluate_model (model, sess, test_data, test_gt)

Testing auc = 0.891710 F1_max = 0.811953 prec = 0.833244 rec = 0.791723
Test Random guess: auc= 0.233054 fmeasure_max= 0.375718 , precision= 0.231313 , recall= 1.000000


In [ ]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
probability_change = np.squeeze(probability_of_pos, axis = 2)

In [ ]:
savemat('../poc/poc'+exp_name+'.mat', {'poc':probability_change})